In [1]:
import json
from urllib import request, parse
import uuid
import urllib.request
import urllib.parse
import io
import os
from PIL import Image
import websocket
from urllib.error import URLError


## 导入prompt模板
comfyUI的模板其实就是它的节点信息json,这里我们加载的这个节点信息文件，是只有一个controlnet模块的节点信息文件，具体内容可以看print的结果

In [2]:
promptJson = 'workflow_api_controlnet.json'

with open(promptJson) as f:
    prompt = json.load(f)

print(str(prompt))

{'3': {'inputs': {'seed': 871461132526194, 'steps': 40, 'cfg': 7.5, 'sampler_name': 'dpmpp_3m_sde_gpu', 'scheduler': 'exponential', 'denoise': 1, 'model': ['18', 0], 'positive': ['15', 0], 'negative': ['15', 1], 'latent_image': ['5', 0]}, 'class_type': 'KSampler'}, '4': {'inputs': {'ckpt_name': 'fudukiMix_v15.safetensors'}, 'class_type': 'CheckpointLoaderSimple'}, '5': {'inputs': {'width': 832, 'height': 1216, 'batch_size': 4}, 'class_type': 'EmptyLatentImage'}, '6': {'inputs': {'text': 'nude,1 girl,(female child:0.3),cuty face,(high resolution detail of human skin texture:1.4,rough skin:1.2),(indirect lighting),pussy', 'clip': ['18', 1]}, 'class_type': 'CLIPTextEncode'}, '7': {'inputs': {'text': '(worst quality, low quality:1.4), (deformed, distorted, disfigured:1.3), (depth of field, bokeh, blurry, blurry background:1.4), manicure,lip,illustration, 2d, 2.5d, painting, cartoons, sketch, watercolor, logo, watermark, double navel, greyscale,', 'clip': ['18', 1]}, 'class_type': 'CLIPText

## 各种参数设置
各种基本参数都在这里设置

In [3]:
# 服务器端口，如果不是本机需要设置成服务器的ip地址
server_address = "127.0.0.1:8188"
client_id = str(uuid.uuid4())

# 种子值，当为0时，随机生成
seed = 0
prompt['3']['inputs']['seed'] = seed
# 使用的sdxl模型
checkPoint = 'fudukiMix_v15.safetensors'
prompt['4']['inputs']['ckpt_name'] = checkPoint
# 批量生成的图片数量
batch_size = 1
prompt['5']['inputs']['batch_size'] = batch_size
# 使用的controlnet模型，这里是使用了一个深度图的controlnet
controlnet = 'models\\t2i-adapter_diffusers_xl_depth_midas.safetensors'
prompt['11']['inputs']['control_net_name'] = controlnet
# controlnet起作用的区间
controlnet_start = 0
controlnet_end = 0.5
prompt['15']['inputs']['start_percent'] = controlnet_start
prompt['15']['inputs']['end_percent'] = controlnet_end
# 使用的lora模型，这里只使用了一个lora
lora_name = 'skirtlift_xl.safetensors'
strength_clip = 0.6
prompt['18']['inputs']['lora_name'] = lora_name
prompt['18']['inputs']['strength_clip'] = strength_clip


In [4]:
# 根据原图的宽高，计算出缩放后的宽高
def changeImageSize(width, height):
    # 最小的宽高为 896
    limited = 896
    minSize = min(width, height)

    # 缩放宽高，使得最窄的边等于limited
    if minSize > limited:
        scale = minSize / limited
        widthScale = int(width / scale)
        heightScale = int(height / scale)
    else:
        scale = limited / minSize
        widthScale = int(width * scale)
        heightScale = int(height * scale)

    # 将两个值都除以64，然后取整,也就是说边长必须是64的倍数
    widthScale = int(widthScale/64) * 64
    heightScale = int(heightScale/64) * 64

    return widthScale, heightScale
# 检测同名图片是否存在
def checkImageExsit(imageDir, imageName):
    for file in os.listdir(imageDir):
        if file.startswith(imageName):
            return True
    return False
# 下面几个方法是发送请求，返回图片的方法
def queue_prompt(prompt):
    while True:
        try:
            p = {"prompt": prompt, "client_id": client_id}
            data = json.dumps(p).encode('utf-8')
            req = urllib.request.Request("http://{}/prompt".format(server_address), data=data)
            return json.loads(urllib.request.urlopen(req, timeout=5).read())
        except URLError:
            print("连接超时，正在重试...")

def get_history(prompt_id):
    with urllib.request.urlopen("http://{}/history/{}".format(server_address, prompt_id)) as response:
        return json.loads(response.read())

def get_image(filename, subfolder, folder_type):
    data = {"filename": filename, "subfolder": subfolder, "type": folder_type}
    url_values = urllib.parse.urlencode(data)
    with urllib.request.urlopen("http://{}/view?{}".format(server_address, url_values)) as response:
        return response.read()

def get_images(ws, prompt):
    prompt_id = queue_prompt(prompt)['prompt_id']
    output_images = {}
    while True:
        out = ws.recv()
        if isinstance(out, str):
            message = json.loads(out)
            if message['type'] == 'executing':
                data = message['data']
                if data['node'] is None and data['prompt_id'] == prompt_id:
                    break #Execution is done
        else:
            continue #previews are binary data

    history = get_history(prompt_id)[prompt_id]
    for o in history['outputs']:
        for node_id in history['outputs']:
            node_output = history['outputs'][node_id]
            if 'images' in node_output:
                images_output = []
                for image in node_output['images']:
                    image_data = get_image(image['filename'], image['subfolder'], image['type'])
                    images_output.append(image_data)
            output_images[node_id] = images_output

    return output_images
# 保存图片的方法  
def saveImages(result, imageDir,imageName,controlnetNumber=0):       
    for node_id in result:
        k = 0
        for image_data in result[node_id]:            
            image = Image.open(io.BytesIO(image_data))
            imagePath = imageDir + "/" + imageName + f"_{k}" + ".png"
            image.save(imagePath) 
            k += 1
    

## 设置输入输出文件夹
这里设置输入输出文件夹

In [5]:
# 子文件夹
subDir = "4.12"
# 原图文件夹
originalDir = f"you_orginal_image_dir/{subDir}"
# 设置图片保存路径
imageDir = f"you_output_image_dir/{subDir}"
# 如果文件夹不存在，创建文件夹
if not os.path.exists(imageDir):
    os.mkdir(imageDir)

## 执行批量生成

In [6]:
# 正面提示词
input_text = '1 girl,cuty face,(high resolution detail of human skin texture:1.4,rough skin:1.2),(indirect lighting)'
prompt['6']['inputs']['text'] = input_text
# 负面提示词
negative_text = '(worst quality, low quality:1.4), (deformed, distorted, disfigured:1.3), (depth of field, bokeh, blurry, blurry background:1.4), manicure,lip,illustration, 2d, 2.5d, painting, cartoons, sketch, watercolor, logo, watermark, double navel, greyscale,'
prompt['7']['inputs']['text'] = negative_text

ws = websocket.WebSocket()
ws.connect("ws://{}/ws?clientId={}".format(server_address, client_id))

# 遍历原图文件夹里所有的图片
for originalImag in os.listdir(originalDir):
        # 如果不是jpg文件，跳过
        if not originalImag.endswith(".jpg"):
            continue
        
        originalImagePath = originalDir + "/" + originalImag
        # 读取原图，以及其大小
        originalImage = Image.open(originalImagePath)
        originalImageWidth = originalImage.size[0]
        originalImageHeight = originalImage.size[1]
        
        # 读取原图片名字，去掉后缀.png
        originalImagName = originalImag.split(".")[0]
        
        print(f"原图名字：{originalImagName}。大小：{originalImageWidth}x{originalImageHeight}")
        width,height = changeImageSize(originalImageWidth, originalImageHeight)
        print(f"缩放后的图片大小是{width}*{height}")
        
        prompt['5']['inputs']['width'] = width
        prompt['5']['inputs']['height'] = height
        prompt['13']['inputs']['image'] = originalImagePath
        
        if checkImageExsit(imageDir, originalImagName):
            print(f"{originalImagName}已经存在，跳过")
            continue
        
        
        images = get_images(ws, prompt)
        
        saveImages(images, imageDir,originalImagName,controlnetNumber=0) 
        

原图名字：019。大小：720x960
缩放后的图片大小是896*1152


原图名字：020。大小：720x960
缩放后的图片大小是896*1152
